# Overview
In this notebook, we apply a parameter estimation method on Kholodenko's model of EGFR signalling pathway. This involves performing parameter estimation, then assessing the quality of the optimized model.

The model that is taken from [Kholodenko et. al (1999)](https://www.sciencedirect.com/science/article/pii/S0021925819518804), and has already been implemented in the rule-based modeling using BioNetGen and also in the
systems biology model specification format (SBML).

The model in Step 4 recovers the Kholodenko's output, but the results of the Kholodenko's model do not fit the experimental data very well, as we can see in the following figure. Moreover, there are some inconsistancies in the parameters which are involved in the thermodynamic restrictions along cyclic pathways in the Kholodenko's model. As a result, in order to discover a set of optimized parameters which not only provides an appropriate fit to the experimental data but also satisfies the detailed balance constraints, I employed a parameter estimation method for the model in Step 4. 

The set of optimized parameters seeks to provide outputs which best follow the experimental data in Kholodenko's paper. 
</ul>
In this figure, you can see the Kholodenko's model output vs experimental data:


</ul>
<br>

![KholodenkoModel.png](KholodenkoModel.png)
<br><br>

## Problem Specification, Import, and Setup

In [ ]:
import petab
import pypesto
import pypesto.optimize as optimize
import pypesto.petab
import pypesto.sample as sample
import pypesto.visualize as visualize
import amici
from pypesto.store import read_from_hdf5, save_to_hdf5
import matplotlib.pyplot as plt
import numpy as np

The parameter estimation problem has also already been formulated in [PEtab](https://github.com/PEtab-dev/PEtab).
The PEtab format is compatible with a variety of tools that are primarily developed within the systems biology community. Here, the [pyPESTO](https://github.com/ICB-DCM/pyPESTO) tool is for parameter estimation. 

The default simulation tool used by pyPESTO is [AMICI](https://github.com/AMICI-dev/AMICI).



In [ ]:
petab_problem = petab.Problem.from_yaml(
    
   "../EGFR/EGFR.yaml"    #state the exact folder contains the yaml file
)
importer = pypesto.petab.PetabImporter(petab_problem)
# import to pypesto
problem = importer.create_problem()
model = importer.create_model(verbose=False)

## Parameter Estimation

A multi-start optimization is used here, to efficiently explore the parameter space for optima. The author's experience with the difficulty of optimizing this problem led her to use the [Fides](https://github.com/fides-dev/fides) optimizer.
The choice of number of starts is problem-specific.

In [ ]:
# create optimizer object which contains all information for doing the optimization
options = {'maxiter':2000}
optimizer = optimize.FidesOptimizer(options=options)
engine = pypesto.engine.MultiProcessEngine()

# do the optimization
result = optimize.minimize(
    problem=problem, optimizer=optimizer, n_starts=200, engine=engine
)

## Visualization and Assessment of Optimization

The first plot here is the waterfall plot, which shows the likelihood function value of the estimated parameter values at the end of each optimization run (start). The runs are ordered by likelihood function value. Generally, a plateau of a few starts suggests a successful optimization with the good global optima found. Other colors are used here to indicate plateaus showing different local optimums.

If the waterfall plot does not show a plateau at the minimum, the bounds can be adjusted (preferably to more realistic bounds), or the optimization can be run with a higher number of starts, or a different optimization method.

In [ ]:
visualize.waterfall(result)

![waterfall](Waterfall.png)

The following figure allows us to compare three sets of the model parameters:

1- The values in the original Kholodenko's paper: Green dots

2- The estimated values without detailed balance constraints: Red dots

3- The estimated values with detailed balance constraints: Black dots

More comparisons for these three methods have been provided in a file, [here](https://github.com/ZarifehHeidariRarani/Energy-Based-Modeling/blob/main/KholodenkoModel_DetailedBalance/Comparing_based_on_DetailedBalanceConstraints.docx)

![Parameters_withoutIC.png](Parameters_withoutIC.png)

We can also conveniently visualize the model fit. This plots the petab visualization using optimized parameters.

In [ ]:
from pypesto.visualize.model_fit import visualize_optimized_model_fit
pp1 = visualize_optimized_model_fit(petab_problem=petab_problem, result=result)

![Optimization](Optimization.png)
<br><br>



### Cooperativity of ShcP in the original Kholodenko model
For original Kholodenko model we have:

#### v13: Unphosphorylated Shc + EGFR
```
K13 = 0.15
```
#### v15: Phosphorylated Shc + EGFR
```
K15 = 0.003
```
### Negative or positive cooperativity of ShcP in the model:

#### v13: Unphosphorylated Shc + EGFR
```
K13 = 7.1e-4
```
#### v15: Phosphorylated Shc + EGFR
```
K15 = 1.48e-7
```
Based on the estimated parameters, it seems that for both sets of parameter values the affinity of Shc for binding to EGFR intensively decreases after phosphorylation.


# Assessment of Maximum Likelihood Estimate

Once optimization appears successful, the maximum likelihood estimate (MLE) can be assessed for parameter and prediction uncertainty.

Parameter uncertainty can be assessed with MCMC sampling.

## MCMC Sampling

Markov chain Monte Carlo (MCMC) sampling is a method of analysing the uncertainty of a parameter estimate. Here,  Parallel Tempering MCMC and adaptive Metropolis algorithm is used.

In [ ]:
mle = result.optimize_result.list[0]['x'] # Maximum likelihood from optimization

sampler = sample.ParallelTemperingSampler(
   internal_sampler=sample.AdaptiveMetropolisSampler(), n_chains = 5
)
result = sample.sample(
    problem = problem, 
    n_samples = 2e6,
    sampler = sampler,
    x0 = mle
)


### Sampling Evaluation

First, we determine the acceptance rate of the MCMC sampling.

In [ ]:
def calc_acceptance(samp_result):
    params = samp_result['trace_x'][0]
    accept = [0 if (params[i, :] == params[i-1, :]).all() else 1 
              for i in range(1, params.shape[0])]
    num_accept = np.sum(accept)
    rate = num_accept / (params.shape[0] - 1)
    return rate

accept_rate = calc_acceptance(result.sample_result)
print("The acceptance rate of MCMC is:  %.2f%%"%(accept_rate*100))

The acceptance rate of MCMC is:  29.59%

We want to monitor the acceptance rate and make sure it is within optimal range. If we accept almost every time, that tells us that each time the chain only jumps a very small step (so that the acceptance ratio is close to 1 every time), which will make the algorithm slow in converging to the stationary distribution.

On the other hand, if the acceptance rate is very low, then that says that the chain got stuck to just a few locations and it takes hundreds of iterations for it to make one jump. For the Metropolis algorithm, an optimal acceptance rate would be something between 10% to 60%. 

For our model, the acceptance rate is within the optimal range.

### Geweke Test

Every Markov chain needs a certain amount of iterations to reach the stationary distribution. Sometimes the chain quickly get to the regions with relative high density, for some situations, especially for multiparameter problems, it usually takes hundreds or thousands of iterations to get there. Iterations obtained before a Markov chain reaches the stationary distribution are called burn-in and can be determined using Geweke test.

In [ ]:
sample.geweke_test(result=result)

Geweke burn-in index: 800000

### Visualize the result
#### Log posterior 
x and y-axis show the iterations and log posterior of function values, respectively.

In [ ]:
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_fval_traces(result,  i_chain=i_chain, stepsize= 10) 

### Chain 1

![fval0](fval0.png)

### Chain 2

![fval1](fval1.png)

### Chain 3

![fval2](fval2.png)

### Chain 4

![fval3](fval3.png)

### Chain 5

![fval4](fval4.png)

The log-posterior chain should be smoothly varying around the maximum. We can say it has been obtained in this sampling.

#### Parameter trajectories 

In [ ]:
n_par = 30
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_parameter_traces(
        result, i_chain=i_chain, par_indices = [4, 7, 8, 10], stepsize=10
    )

### Chain 1

![ParamTrace0](ParamTrace0.png)

### Chain 2

![ParamTrace1](ParamTrace1.png)

### Chain 3

![ParamTrace2](ParamTrace2.png)

### Chain 4

![ParamTrace3](ParamTrace3.png)

### Chain 5

![ParamTrace4](ParamTrace4.png)

We also can plot e.g. kernel density estimates or histograms.

### Chain 1

![MCMC0](MCMC0.png)

### Chain 2

![MCMC1](MCMC1.png)

### Chain 3

![MCMC2](MCMC2.png)

### Chain 4

![MCMC3](MCMC3.png)

### Chain 5

![MCMC4](MCMC4.png)

For having a better view, histograms for several parameters are shown:

### Chain 1

![MCMC0_4](MCMC0_4.png)

### Chain 2

![MCMC1_4](MCMC1_4.png)

### Chain 3

![MCMC2_4](MCMC2_4.png)

### Chain 4

![MCMC3_4](MCMC3_4.png)

### Chain 5

![MCMC4_4](MCMC4_4.png)

## Gelman-Rubin Convergence Diagnostic

Gelman and Rubin diagnostics is introduce in Gelman and Rubin (1992) to compare several chains. This approach is based on the analysis of variance. Approximate convergence is diagnosed when the variance between the different chain, is no larger than the variance within each individual chain.

Then, potential scale reduction factor (PSRF) is calculated for each of the parameters.

```
PSRF = 
[1.00372036 1.00023034 1.0070337  1.00435328 1.00109211 1.00004674
 1.00006971 1.00257258 1.00719548 1.00490399 1.00326198 1.00265316
 1.00259543 1.01882634 1.00010566 1.00099842 1.00043986 1.00371582
 1.00126627 1.00000426 1.00382374 1.00297572 1.00459077 1.00073319
 1.00094755 1.00034744 1.00265541 1.00228857 1.00246688 1.00025032
 1.00127031 1.00000288 1.00026312 1.00000407 1.00003315 1.0020304
 1.00140989 1.00133685 1.00035036 1.00123513 1.00015638 1.00314101
 1.00014624 1.00864126 1.16647382]
```

Since the values of the matrix elements is close to 1 (smaller than 1.2), the theory claims that they should all eventually converge to the stationary distribution. The Gelman-Rubin statistic is a ratio, and hence unit free, making it a simple summary for any MCMC sampler. Therefore, it can be a useful tool for monitoring a chain before any specific decisions about what kinds of inferences will be made from the model.